In [111]:
import pandas as pd
import numpy as np
from semopy import Model
import matplotlib.pyplot as plt

In [112]:
# Load the dataset
data = pd.read_csv("/Users/stageacomeback/Desktop/Gerard Lee/Job [MATH RA]/CLPM & RI-CLPM/Gerard's Working Environment/Model Results/Data Cleaning & FInal CLPM Model/Quantile-based Outlier Detection/7th Trial.csv")

In [113]:
# Define the Cross-Lagged Panel Model (CLPM) in semopy syntax
model_desc = """
R3TTF ~ R2TTF + R2OWB + R2AU
R3OWB ~ R2OWB + R2TTF + R2AU
R3AU ~ R2AU + R2TTF + R2OWB

# Covariances at Wave 2
R2TTF ~~ R2OWB + R2AU
R2OWB ~~ R2AU

# Covariances at Wave 3
R3TTF ~~ R3OWB + R3AU
R3OWB ~~ R3AU
"""

In [114]:
# Create the model
model = Model(model_desc)
model

In [115]:
# Fit the model to the dataset
try:
    model.fit(data)
    print("Model Summary:")
    print(model.inspect())
except Exception as e:
    print(f"Model fitting failed on the original dataset: {e}")
    exit()

Model Summary:
     lval  op   rval  Estimate  Std. Err   z-value       p-value
0   R3TTF   ~  R2TTF  0.246378  0.084228  2.925140  3.443012e-03
1   R3TTF   ~  R2OWB  0.205050  0.088365  2.320484  2.031470e-02
2   R3TTF   ~   R2AU  0.128886  0.064428  2.000465  4.545010e-02
3   R3OWB   ~  R2OWB  0.111667  0.071048  1.571717  1.160163e-01
4   R3OWB   ~  R2TTF  0.365024  0.067721  5.390086  7.042404e-08
5   R3OWB   ~   R2AU  0.151074  0.051802  2.916390  3.541080e-03
6    R3AU   ~   R2AU  0.339596  0.065049  5.220639  1.783066e-07
7    R3AU   ~  R2TTF  0.274621  0.085040  3.229330  1.240805e-03
8    R3AU   ~  R2OWB  0.235387  0.089217  2.638377  8.330402e-03
9   R2TTF  ~~  R2OWB  0.295878  0.030881  9.581232  0.000000e+00
10  R2TTF  ~~   R2AU  0.195997  0.050221  3.902704  9.512394e-05
11  R2OWB  ~~   R2AU  0.246994  0.045996  5.369923  7.877036e-08
12  R3TTF  ~~  R3OWB  0.171738  0.036862  4.658879  3.179357e-06
13  R3TTF  ~~   R3AU  0.077308  0.043701  1.769040  7.688723e-02
14  R3TTF 

In [116]:
# Monte Carlo Simulation for repeated sampling
np.random.seed(123)  # For reproducibility
n_simulations = 1000
simulated_errors = []

In [117]:
for i in range(n_simulations):
    # Generate a bootstrap sample
    bootstrap_sample = data.sample(n=len(data), replace=True)

    try:
        # Fit the model to the bootstrap sample
        model.fit(bootstrap_sample)
        
        # Extract standard errors from the model
        std_errors = model.inspect("se")
        
        if std_errors is not None:
            simulated_errors.append(std_errors)
    except Exception as e:
        # Skip iterations where model fitting fails
        print(f"Model fitting failed in simulation {i+1}: {e}")
        continue

In [118]:
# Check if we have valid results
if len(simulated_errors) == 0:
    print("No valid standard errors were obtained during the simulation. Check the model or data.")
else:
    # Convert simulated standard errors to a DataFrame
    simulated_errors_df = pd.DataFrame(simulated_errors)

    # Identify outliers based on the 90 percentile
    outliers = simulated_errors_df[simulated_errors_df > simulated_errors_df.quantile(0.9)].dropna(how="all")

    # Print simulation results and outliers
    print("Summary of Simulated Standard Errors:")
    print(simulated_errors_df.describe())
    print("Outliers detected:")
    print(outliers)

    # Visualize the standard error distribution
    plt.hist(simulated_errors_df.values.flatten(), bins=30, color="skyblue", edgecolor="white")
    plt.title("Distribution of Simulated Standard Errors")
    plt.xlabel("Standard Error")
    plt.ylabel("Frequency")
    plt.show()

No valid standard errors were obtained during the simulation. Check the model or data.
